In [9]:
import operator


def preprocess(sentence):
    preprocessed_sentence = sentence.strip().split(" ")
    return preprocessed_sentence

def indexing(file_name):
    file_tokens_pairs = []    
    lines = open(file_name, "r", encoding="utf8").readlines()
    for line in lines:
        tokens = preprocess(line) # lines의 각 아이템들의 앞뒤 공백을 제거하고 아이템 내의 공백을 기준으로 나눔
        file_tokens_pairs.append(tokens) # 빈 리스트에 tokens, 즉, 한 단어씩을 추가
    return file_tokens_pairs


def cals_similarity(preprocessed_query, preprocessed_sentences):
    score_dict = {}
    
    for i in range(len(preprocessed_sentences)): # 찾을 문장의 대소문자 구분없이 처리하기 위함
        sentence = []
        for word in preprocessed_sentences[i]:
            sentence.append(word.lower()) # 소문자로 변환하여 sentence에 저장
        file_token_set = set(sentence)
        all_tokens = preprocessed_query | file_token_set # all_tokens는 쿼리토큰셋과 파일토큰셋의 합집합
        same_tokens = preprocessed_query & file_token_set # same_tokens는 쿼리토큰셋과 파일토큰셋의 교집합
        similarity = len(same_tokens) / len(all_tokens) # 유사도를 계산하기 위해 일치하는 토큰 수를 전체 토큰 수로 나눔
        similarity = round(similarity, 4) # 유사도 점수가 너무 길어지는걸 방지하기 위해 소수점 4자리까지 나타내도록 수정
        score_dict[i] = similarity
    return score_dict


# 1. Indexing
## https://github.com/jungyeul/korean-parallel-corpora
file_name = "jhe-koen-dev.en"
file_tokens_pairs = indexing(file_name)

# 2. Input the query
query = input("영어 쿼리를 입력하세요.").lower()
preprocessed_query = preprocess(query) # query의 앞뒤 공백을 제거하고, query 내의 공백을 기준으로 나눔
query_token_set = set(preprocessed_query) # 위의 preprocessed_query에서 중복된 항목을 하나로 병합

# 3. Calculate similarities based on a same token set
# cals_similarity 함수를 호출하여 query_token_set과 file_tokens_set 사이의 유사도를 계산하고 결과를 score_dict에 저장
score_dict = cals_similarity(query_token_set, file_tokens_pairs)

# 4. Sort the similarity list
# score_dict를 유사도 점수에 따라 내림차순(기본세팅인 오름차순의 reverse=True)으로 정렬하여 sorted_score_list에 저장
sorted_score_list = sorted(score_dict.items(), key = operator.itemgetter(1), reverse=True)

# 5. Print the result
# 가장 높은 유사도가 0.0인 경우 "There is no similar sentence." 출력
if sorted_score_list[0][1] == 0.0:
    print("There is no similar sentence.")
else: # 가장 높은 유사도가 0.0이 아닌 경우
    print("rank", "Index", "score", "sentence", sep = "\t") # rank, Index, score, sentence를 \t로 구분
    rank = 1
    for i, score in sorted_score_list:
        # 결과를 rank, index, score 및 해당 문장으로 출력, 각 요소는 \t로 구분
        print(rank, i, score, ' '.join(file_tokens_pairs[i]), sep = "\t")
        if rank == 10: # rank가 10일 경우
            break # 반복문 탈출
        rank = rank + 1

영어 쿼리를 입력하세요.I am loopy
rank	Index	score	sentence
1	251	0.3333	I am nine years old.
2	623	0.1818	I am sorry for my late reply to your letter.
3	160	0.1667	I just wanted you to know how proud I am of Bobby.
4	405	0.1429	I felt sorry for it.
5	516	0.1429	I just got your flower.
6	388	0.125	I even have my feet pickled!
7	556	0.125	I'm glad I let it go.
8	718	0.125	I saw an accident this morning.
9	62	0.1111	I insisted he take a hearing test.
10	383	0.1111	I don't want to shut them out."
